# Quick Maps Exploration

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
path = '../input/geolifeclef-2022-lifeclef-2022-fgvc9'
metadata_folder = '/metadata/'
observations_folder = '/observations/'

In [ ]:
species = pd.read_csv(path + metadata_folder + 'species_details.csv', sep=';')
train_val_fr = pd.read_csv(path + observations_folder + 'observations_fr_train.csv', sep=';')
train_val_fr['location'] = 'France'
train_val_us = pd.read_csv(path + observations_folder + 'observations_us_train.csv', sep=';')
train_val_us['location'] = 'USA'
train_val = train_val_fr.append(train_val_us)

train = train_val[train_val['subset'] == 'train'].drop(columns='subset')
validation = train_val[train_val['subset'] == 'val'].drop(columns='subset')

train = train.merge(species, on='species_id', how='left')
train.sample(5)

In [ ]:
geom = gpd.points_from_xy(x = train['longitude'], y=train['latitude'], crs=4326)
train_geo = gpd.GeoDataFrame(train, geometry=geom)

Filter to France only

In [ ]:
train_geo_fr = train_geo[train_geo['location']=='France']

### Map by Kingdom

In [ ]:
train_geo_fr['GBIF_kingdom_name'].value_counts(dropna=False).plot(kind='bar', color=['green', 'red'], title='Count by Kingdom (France)');

In [ ]:
kingdom_params = dict(column="GBIF_kingdom_name", tooltip=True, popup=True, cmap=['red', 'green'], marker_kwds=dict(radius=3.5, fill=True), style_kwds=dict(stroke=False))
train_geo_fr.sample(12000).explore(**kingdom_params)

### Map by Families (3 Most Common Families)

For animals:

In [ ]:
animalia = train_geo[train_geo['GBIF_kingdom_name'] == 'Animalia']
animalia_vc = animalia['GBIF_family_name'].value_counts(dropna=False).head(3)
animalia_3 = animalia[animalia['GBIF_family_name'].isin(animalia_vc.index)]

In [ ]:
animalia_family_params = dict(column="GBIF_family_name", tooltip=True, popup=True, cmap='jet', marker_kwds=dict(radius=3.5, fill=True), style_kwds=dict(stroke=False), legend_kwds=dict(caption='Animals Family names'))
animalia_3[animalia_3['location']=='France'].explore(**animalia_family_params)

For plants:

In [ ]:
Plantae = train_geo[train_geo['GBIF_kingdom_name'] == 'Plantae']
Plantae_vc = Plantae['GBIF_family_name'].value_counts(dropna=False).head(3)
Plantae_3 = Plantae[Plantae['GBIF_family_name'].isin(Plantae_vc.index)]

In [ ]:
plantae_family_params = dict(column="GBIF_family_name", tooltip=True, popup=True, marker_kwds=dict(radius=3.0, fill=True), style_kwds=dict(stroke=False), legend_kwds=dict(caption='Plants Family names'))
Plantae_3[Plantae_3['location']=='France'].sample(10000).explore(**plantae_family_params)
#sampled 10,000 for faster visualization

### Map by Species (display 30 most common species)

In [ ]:
most_30_spieces = train_geo_fr['species_id'].value_counts()[:30].index
species_params = dict(column="species_id", tooltip=True, popup=True, marker_kwds=dict(radius=3.5, fill=True), style_kwds=dict(stroke=False))
train_geo_fr[train_geo_fr['species_id'].isin(most_30_spieces)].sample(10000).explore(**species_params)
#sampled 10,000 for faster visualization